In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymysql

In [2]:
def getEvent(soup, match_teams, score, ha, goal_events):
    for i in soup:
        shoter = i.find('div', class_='fi-p__n').find('span', class_='fi-p__nShorter').text.strip()
        time = i.find('span', class_='fi-mh__scorer__minute').text.strip().replace(',', '').replace('\n', '').replace('\r', '')
        label = i.find('span', class_='fi-mh__scorer__label')
        label = '' if label == None else label.text.strip()

        if shoter.__contains__('@'):
            continue

#         print(('%s, %s %s' % (shoter, time, label)).strip())
        goal_events = goal_events.append({'home_team': match_teams[0], 'away_team': match_teams[1], 'score': score, 'goal_team': match_teams[0] if ha == 1 else match_teams[1], 'shoter': shoter, 'time': time, 'label': label}, ignore_index=True)
        
    return goal_events

In [3]:
def getGoal(urlstr, goal_events):
    r = requests.get(urlstr)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    match_teams = soup.title.text.split(' - ')[2:4]
    score = soup.find('span', class_='fi-s__scoreText').text.strip()
    
#     print(':'.join(match_teams))
#     print('score: %s' % score)
    
    home_event = soup.find_all('div', class_ = 'fi-mh__scorers__home')[0].find_all('div', class_='fi-mh__scorer__event')
    
    if home_event.__len__() > 0:
#         print('Home')
        goal_events = getEvent(home_event, match_teams, score, 1, goal_events)
    
    away_event = soup.find_all('div', class_ = 'fi-mh__scorers__away')[0].find_all('div', class_='fi-mh__scorer__event')
    
    if away_event.__len__() > 0:
#         print('Away')
        goal_events = getEvent(away_event, match_teams, score, 2, goal_events)
        
    return goal_events

In [4]:
web_urls =[ 'https://www.fifa.com/worldcup/matches/?#groupphase',
          'https://www.fifa.com/worldcup/matches/?#knockoutphase']

goal_events = pd.DataFrame(columns=['home_team', 'away_team', 'score', 'goal_team', 'shoter', 'time', 'label'])

for url in web_urls[1:]:
    res = requests.get(url)
    html_txt = res.text
    soup = BeautifulSoup(html_txt, 'html.parser')
    match_links = soup.findAll("a", {"class": "fi-mu__link"})
    
    
    for match_link in match_links:
        goal_events = getGoal('https://www.fifa.com%s%s' % (match_link['href'], '#match-info'), goal_events)
#         print()

In [157]:
goal_events.sort_values(['home_team', 'away_team', 'time'])

,home_team,away_team,score,goal_team,shoter,time,label
50,Argentina,Croatia,0-3,Croatia,REBIĆ,53',
49,Argentina,Croatia,0-3,Croatia,MODRIĆ,80',
48,Argentina,Croatia,0-3,Croatia,RAKITIĆ,90'+1',
16,Argentina,Iceland,1-1,Argentina,KUN AGUERO,19',
17,Argentina,Iceland,1-1,Iceland,FINNBOGASON,23',
98,Australia,Peru,0-2,Peru,CARRILLO,18',
97,Australia,Peru,0-2,Peru,GUERRERO,50',
162,Belgium,England,2-0,Belgium,MEUNIER,4',
161,Belgium,England,2-0,Belgium,E. HAZARD,82',
141,Belgium,Japan,3-2,Japan,HARAGUCHI,48',


In [ ]:
goal_events[]

In [189]:
goal_events[:10]

,home_team,away_team,score,goal_team,shoter,time,label
0,Russia,Saudi Arabia,5-0,Russia,GAZINSKII,12',
1,Russia,Saudi Arabia,5-0,Russia,GOLOVIN,90'+4',
2,Russia,Saudi Arabia,5-0,Russia,CHERYSHEV,43',
3,Russia,Saudi Arabia,5-0,Russia,CHERYSHEV,90'+1',
4,Russia,Saudi Arabia,5-0,Russia,DZYUBA,71',
5,Egypt,Uruguay,0-1,Uruguay,J.M. GIMENEZ,89',
6,Morocco,IR Iran,0-1,IR Iran,BOUHADDOUZ,90'+5',OG
7,Portugal,Spain,3-3,Portugal,RONALDO,4',PEN
8,Portugal,Spain,3-3,Portugal,RONALDO,44',
9,Portugal,Spain,3-3,Portugal,RONALDO,88',


In [166]:
countries = {'russia':'1',
            'saudi arabia':'2',
            'egypt':'3',
            'uruguay':'4',
            'portugal':'5',
            'spain':'6',
            'morocco':'7',
            'ir iran':'8',
            'france':'9',
            'australia':'10',
            'peru':'11',
            'denmark':'12',
            'argentina':'13',
            'iceland':'14',
            'croatia':'15',
            'nigeria':'16',
            'brazil':'17',
            'switzerland':'18',
            'costa rica':'19',
            'serbia':'20',
            'germany':'21',
            'mexico':'22',
            'sweden':'23',
            'korea republic':'24',
            'belgium':'25',
            'panama':'26',
            'tunisia':'27',
            'england':'28',
            'poland':'29',
            'senegal':'30',
            'colombia':'31',
            'japan':'32'}

In [160]:
goal_events.columns

Index(['home_team', 'away_team', 'score', 'goal_team', 'shoter', 'time',
       'label'],
      dtype='object')

In [183]:
def get_connection():
    CONN = pymysql.connect(host='madigun.asuscomm.com', user='madigun', password='ehenr1163', db='football_worldcup', charset="utf8")
    return CONN

def close():
    CONN.commit()
    CONN.close()
CONN = get_connection()

In [221]:
def get_match_id(home_id, away_id):
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    
    find_sql = '''select id from wc_match where home_team_id=%s and away_team_id=%s'''
    curs.execute(find_sql, (home_id, away_id))
    result = curs.fetchone()
    return result

def get_player_id(country_id, name):
#     print(str(country_id)+';;;;'+str(name))
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    
    find_sql = '''select * from player where team_id=%s and lower(name) like %s'''
    curs.execute(find_sql, (country_id, '%'+name+'%'))
    result = curs.fetchone()
    return result  

def save_stat(player_stat):
    print(player_stat)
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    insert_sql = """insert into player_stat (player_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (player_stat['player_id'], player_stat['match_id'], player_stat['stat_type'], player_stat['stat_value'], player_stat['half'],))
    CONN.commit()

In [222]:
uninserted_rows = []

for ind, row in goal_events.iterrows():
    home_id = countries[row['home_team'].lower()]
    away_id = countries[row['away_team'].lower()]
    match_id = get_match_id(home_id, away_id)
#     print(match_id)
    
    name = row['shoter'].lower().strip()
    if name.find('JØRGENSEN'.lower())>=0:
        if name.split('.')[0]=='m':
            name='Mathias'
        else:
            name='Nicolai'
            
    if name.find('.')>=0:
        name = max(name.split('.'), key=len)
    if name.find(' ')>=0:
        name = max(name.split(' '), key=len)
    if name.find('-')>=0:
        name = max(name.split('-'), key=len)            

    if row['label']!='':
        if row['label']=='PEN':
            stat_type = 'TSCORE_PEN'
            goal_team_id = countries[row['goal_team'].lower()]
        elif row['label']=='OG':
            stat_type = 'TSCORE_OG'
            goal_team_id = countries[row['home_team'].lower() if row['goal_team'].lower()!=row['home_team'].lower() else row['away_team'].lower()]
    else:
        stat_type = 'TSCORE'
        goal_team_id = countries[row['goal_team'].lower()]
           
#     print(goal_team_id)
    shoter_id = get_player_id(goal_team_id, name.replace('ii', 'y'))
    print('%s::: %s' % (row['shoter'].lower(), shoter_id['id']))
    
    save_stat({'player_id':shoter_id['id'],
               'match_id':match_id['id'],
              'stat_type':stat_type,
              'stat_value':row['time'],
              'half':'total'})

gazinskii::: 19
{'player_id': 19, 'match_id': 1, 'stat_type': 'TSCORE', 'stat_value': "12'", 'half': 'total'}
golovin::: 23
{'player_id': 23, 'match_id': 1, 'stat_type': 'TSCORE', 'stat_value': "90'+4'", 'half': 'total'}
cheryshev::: 26
{'player_id': 26, 'match_id': 1, 'stat_type': 'TSCORE', 'stat_value': "43'", 'half': 'total'}
cheryshev::: 26
{'player_id': 26, 'match_id': 1, 'stat_type': 'TSCORE', 'stat_value': "90'+1'", 'half': 'total'}
dzyuba::: 28
{'player_id': 28, 'match_id': 1, 'stat_type': 'TSCORE', 'stat_value': "71'", 'half': 'total'}
j.m. gimenez::: 44
{'player_id': 44, 'match_id': 2, 'stat_type': 'TSCORE', 'stat_value': "89'", 'half': 'total'}
bouhaddouz::: 111
{'player_id': 111, 'match_id': 4, 'stat_type': 'TSCORE_OG', 'stat_value': "90'+5'", 'half': 'total'}
ronaldo::: 75
{'player_id': 75, 'match_id': 3, 'stat_type': 'TSCORE_PEN', 'stat_value': "4'", 'half': 'total'}
ronaldo::: 75
{'player_id': 75, 'match_id': 3, 'stat_type': 'TSCORE', 'stat_value': "44'", 'half': 'total'